In [52]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 MB 5.2 MB/s eta 0:00:00m eta 0:00:010:00:01


In [80]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [68]:
df = pd.read_csv('demographic.csv')
df = df.drop('index', axis=1)
df

/tmp/ipykernel_10236/466465776.py:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('demographic.csv')


,flag_bad,de_age,de_gender,de_date_joined,de_num_friends,de_monthly_salary,de_employment_type,de_employment_duration,de_accomodation_type,de_education,...,de_children,ph_total_contacts,ph_other_device_info,ph_call_log_stats,ph_country_code,ph_app_list,fb_gender,fb_dob,fb_last_updated_date,fb_relation
0,0,26,1,08/04/2018 14:50,2535,3000000,1,51,4,2,...,0,66,"{""device_codename"": ""HM2014817"", ""brand"": ""Xia...","{""percent_incoming_nighttime"": 1.83, ""percent_...",id,"['com.google.android.music', 'com.facebook.orc...",male,11/23/1991,2017-12-21T04:38:00+0000,Married
1,0,22,1,02/01/2018 17:17,685,4000000,1,28,4,4,...,0,69,"{""device_codename"": ""A1601"", ""brand"": ""OPPO""}","{""percent_incoming_nighttime"": 0.0, ""percent_o...",id,"['com.google.android.instantapps.supervisor', ...",male,01/20/1995,2017-11-25T17:23:09+0000,NaN
2,1,32,1,01/01/2018 16:39,1765,7000000,1,35,4,4,...,2,315,"{""device_codename"": ""A1601"", ""brand"": ""OPPO""}","{""percent_incoming_nighttime"": 6.65, ""percent_...",id,"['com.tencent.ibg.joox', 'com.mrgames.larvaact...",male,10/13/1985,2017-12-23T18:05:57+0000,Married
3,0,31,2,04/01/2018 22:36,407,20000000,3,37,4,3,...,2,84,"{""device_codename"": ""A1601"", ""brand"": ""OPPO""}","{""percent_incoming_nighttime"": 0.0, ""percent_o...",id,"['com.kfit.fave', 'app.bpjs.mobile', 'com.redd...",female,07/27/1986,2017-12-05T09:37:19+0000,Married
4,0,27,1,02/01/2018 18:12,3064,3000000,1,36,4,3,...,0,0,"{""device_codename"": ""1609"", ""brand"": ""vivo""}","{""percent_incoming_nighttime"": 0.0, ""percent_o...",id,"['com.lenovo.anyshare.gps', 'com.UCMobile.intl...",male,07/05/1990,2017-12-03T09:09:48+0000,In a relationship
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41017,0,27,2,11/07/2018 22:23,997,5000000,2,51,4,2,...,2,1026,"{""device_codename"": ""A1601"", ""brand"": ""OPPO""}","{""percent_incoming_nighttime"": 5.66, ""percent_...",id,"['com.instagram.android', 'com.google.android....",NaN,NaN,NaN,NaN
41018,0,26,1,11/07/2018 20:28,955,3000000,2,24,4,2,...,0,0,"{""device_codename"": ""nikel"", ""brand"": ""Xiaomi""}","{""percent_incoming_nighttime"": 10.71, ""percent...",id,"['com.youmusic.magictiles', 'com.ayopop', 'com...",NaN,NaN,NaN,NaN
41019,0,22,2,11/07/2018 23:33,881,3000000,1,9,4,4,...,0,272,"{""device_codename"": ""a33g"", ""brand"": ""samsung""}","{""percent_incoming_nighttime"": 0.0, ""percent_o...",id,"['com.grabtaxi.passenger', 'jp.co.translimit.b...",female,10/02/1995,2017-10-25T08:05:13+0000,NaN
41020,0,29,2,11/07/2018 22:03,540,6000000,1,51,4,2,...,0,0,"{""device_codename"": ""santoni"", ""brand"": ""Xiaomi""}","{""percent_incoming_nighttime"": 1.27, ""percent_...",id,"['cn.wps.moffice_eng', 'com.vuclip.viu', 'com....",NaN,NaN,NaN,NaN


In [123]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41022 entries, 0 to 41021
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   flag_bad                41022 non-null  int64 
 1   de_age                  41022 non-null  int64 
 2   de_gender               41022 non-null  int64 
 3   de_date_joined          41022 non-null  object
 4   de_num_friends          41022 non-null  int64 
 5   de_monthly_salary       41022 non-null  int64 
 6   de_employment_type      41022 non-null  int64 
 7   de_employment_duration  41022 non-null  int64 
 8   de_accomodation_type    41022 non-null  int64 
 9   de_education            41022 non-null  int64 
 10  de_marital_status       41022 non-null  int64 
 11  de_children             41022 non-null  int64 
 12  ph_total_contacts       41022 non-null  int64 
 13  ph_other_device_info    41022 non-null  object
 14  ph_call_log_stats       38935 non-null  object
 15  ph

In [124]:
df.isna().sum()/len(df)

flag_bad                  0.000000
de_age                    0.000000
de_gender                 0.000000
de_date_joined            0.000000
de_num_friends            0.000000
de_monthly_salary         0.000000
de_employment_type        0.000000
de_employment_duration    0.000000
de_accomodation_type      0.000000
de_education              0.000000
de_marital_status         0.000000
de_children               0.000000
ph_total_contacts         0.000000
ph_other_device_info      0.000000
ph_call_log_stats         0.050875
ph_country_code           0.000609
ph_app_list               0.000000
fb_gender                 0.036883
fb_dob                    0.235313
fb_last_updated_date      0.035834
fb_relation               0.645044
dtype: float64

Check balanceness of data

In [70]:
df.flag_bad.value_counts()/len(df)

flag_bad
0    0.870313
1    0.129687
Name: count, dtype: float64

Not really unbalanced

In [71]:
df.de_accomodation_type.value_counts()

de_accomodation_type
4    41022
Name: count, dtype: int64

In [72]:
df.ph_country_code.value_counts()

ph_country_code
id    40984
us        3
sg        3
my        2
jp        2
tw        1
kh        1
th        1
Name: count, dtype: int64

Dont use these data, it is not varied enough

# Baseline Modelling

In [73]:
df_bs.de_employment_type.value_counts()

de_employment_type
1    29566
2     6621
3     4833
4        2
Name: count, dtype: int64

In [74]:
df_bs.de_education.value_counts()

de_education
2    24589
4     8258
3     5758
5     1561
1      856
Name: count, dtype: int64

In [75]:
df_bs.de_children.value_counts()

de_children
0    23095
1     8921
2     7249
3     1446
4      311
Name: count, dtype: int64

In [76]:
df_bs.de_marital_status.value_counts()

de_marital_status
1    20154
2    20020
3      500
4      348
Name: count, dtype: int64

In [259]:
df_bs = df[[
    'flag_bad',
    'de_age',
    'de_gender',
    'de_num_friends',
    'de_monthly_salary',
    'de_employment_type',
    'de_employment_duration',
    'de_education',
    'de_marital_status',
    'de_children',
    'ph_total_contacts',
]].copy()
df_bs

,flag_bad,de_age,de_gender,de_num_friends,de_monthly_salary,de_employment_type,de_employment_duration,de_education,de_marital_status,de_children,ph_total_contacts
0,0,26,1,2535,3000000,1,51,2,2,0,66
1,0,22,1,685,4000000,1,28,4,1,0,69
2,1,32,1,1765,7000000,1,35,4,2,2,315
3,0,31,2,407,20000000,3,37,3,2,2,84
4,0,27,1,3064,3000000,1,36,3,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
41017,0,27,2,997,5000000,2,51,2,2,2,1026
41018,0,26,1,955,3000000,2,24,2,1,0,0
41019,0,22,2,881,3000000,1,9,4,1,0,272
41020,0,29,2,540,6000000,1,51,2,1,0,0


In [260]:
df_bs.de_gender = df_bs.de_gender.astype(str)
df_bs.de_employment_type = df_bs.de_employment_type.astype(str)
df_bs.de_education = df_bs.de_education.astype(str)
df_bs.de_marital_status = df_bs.de_marital_status.astype(str)
df_bs.de_children = df_bs.de_children.astype(str)
df_bs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41022 entries, 0 to 41021
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   flag_bad                41022 non-null  int64 
 1   de_age                  41022 non-null  int64 
 2   de_gender               41022 non-null  object
 3   de_num_friends          41022 non-null  int64 
 4   de_monthly_salary       41022 non-null  int64 
 5   de_employment_type      41022 non-null  object
 6   de_employment_duration  41022 non-null  int64 
 7   de_education            41022 non-null  object
 8   de_marital_status       41022 non-null  object
 9   de_children             41022 non-null  object
 10  ph_total_contacts       41022 non-null  int64 
dtypes: int64(6), object(5)
memory usage: 3.4+ MB


In [261]:
df_bs = pd.get_dummies(df_bs)

In [262]:
y = df_bs.flag_bad
X = df_bs.drop('flag_bad', axis=1)

In [263]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [264]:
model = XGBClassifier()
model.fit(X_train, y_train)

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [265]:
y_pred = model.predict(X_test)

In [266]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.99      0.93      7167
           1       0.31      0.02      0.04      1038

    accuracy                           0.87      8205
   macro avg       0.59      0.51      0.49      8205
weighted avg       0.80      0.87      0.82      8205



# Second Model

Employment type 4 assign to 3 because
1. Its not provided in documentation, can be misfill
2. Only 2
3. 3 is the nearest

And we also feature eningeering month of join date

Adding fb gender also and put unstated to nan

Adding joining age feature engineered from join date assume today is the prediction. 

In [197]:
df.fb_relation.value_counts()

fb_relation
Married                                6327
Single                                 4495
In a relationship                      1847
Engaged                                 812
It's complicated                        362
In an open relationship                 158
Married (Pending)                       154
In a domestic partnership               118
Separated                               110
Divorced                                 59
Widowed                                  57
In a relationship (Pending)              28
Engaged (Pending)                        22
In a civil union                          5
In an open relationship (Pending)         3
In a domestic partnership (Pending)       3
It's complicated (Pending)                1
Name: count, dtype: int64

In [300]:
df_bs = df[[
    'flag_bad',
    'de_age',
    'de_gender',
    # 'de_date_joined',
    'de_num_friends',
    'de_monthly_salary',
    'de_employment_type',
    'de_employment_duration',
    'de_education',
    'de_marital_status',
    'de_children',
    'ph_total_contacts',
    # 'fb_gender',
    # 'fb_relation'
]].copy()
df_bs

,flag_bad,de_age,de_gender,de_num_friends,de_monthly_salary,de_employment_type,de_employment_duration,de_education,de_marital_status,de_children,ph_total_contacts
0,0,26,1,2535,3000000,1,51,2,2,0,66
1,0,22,1,685,4000000,1,28,4,1,0,69
2,1,32,1,1765,7000000,1,35,4,2,2,315
3,0,31,2,407,20000000,3,37,3,2,2,84
4,0,27,1,3064,3000000,1,36,3,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
41017,0,27,2,997,5000000,2,51,2,2,2,1026
41018,0,26,1,955,3000000,2,24,2,1,0,0
41019,0,22,2,881,3000000,1,9,4,1,0,272
41020,0,29,2,540,6000000,1,51,2,1,0,0


In [301]:
df_bs.de_employment_type = df_bs.de_employment_type.replace(4,3)

In [302]:
# df_bs['de_joining_age'] = (pd.Timestamp.today() - pd.to_datetime(df_bs.de_date_joined, dayfirst=True)).dt.days

In [303]:
# df_bs['de_month'] = pd.to_datetime(df_bs.de_date_joined, dayfirst=True).dt.month
# df_bs = df_bs.drop('de_date_joined', axis=1)

In [304]:
# df_bs.fb_gender = df_bs.fb_gender.fillna("unstated")

In [305]:
df_bs.de_gender = df_bs.de_gender.astype(str)
df_bs.de_employment_type = df_bs.de_employment_type.astype(str)
df_bs.de_education = df_bs.de_education.astype(str)
df_bs.de_marital_status = df_bs.de_marital_status.astype(str)
df_bs.de_children = df_bs.de_children.astype(str)
# df_bs.de_month = df_bs.de_month.astype(str)
df_bs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41022 entries, 0 to 41021
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   flag_bad                41022 non-null  int64 
 1   de_age                  41022 non-null  int64 
 2   de_gender               41022 non-null  object
 3   de_num_friends          41022 non-null  int64 
 4   de_monthly_salary       41022 non-null  int64 
 5   de_employment_type      41022 non-null  object
 6   de_employment_duration  41022 non-null  int64 
 7   de_education            41022 non-null  object
 8   de_marital_status       41022 non-null  object
 9   de_children             41022 non-null  object
 10  ph_total_contacts       41022 non-null  int64 
dtypes: int64(6), object(5)
memory usage: 3.4+ MB


In [306]:
df_bs = pd.get_dummies(df_bs)

In [307]:
y = df_bs.flag_bad
X = df_bs.drop('flag_bad', axis=1)

In [308]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [309]:
model = XGBClassifier()
model.fit(X_train, y_train)

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [310]:
y_pred = model.predict(X_test)

In [311]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.99      0.93      7167
           1       0.37      0.03      0.05      1038

    accuracy                           0.87      8205
   macro avg       0.62      0.51      0.49      8205
weighted avg       0.81      0.87      0.82      8205



# EDA